In [8]:
from Preprocessing.dataloader import *
from torch.utils.data import DataLoader

speed_volume_perturb=False
spec_augment=False
sr = 16000
audio_config = dict(sample_rate=sr,
                          window_size=.02,
                          window_stride=0.01,
                          window='hamming',
                          noise_dir=None,
                          noise_prob=0.4,
                          noise_levels=(0.0, 0.5))
directory = './Data/Hip-Hop/'
out_dir = './Output'
pos_dir = './Data/spectrogram_pos'
neg_dir = None

# save npy files 
if not path.exists(pos_dir):
    os.mkdir(pos_dir)
    if path.exists(out_dir):
        ! rm -r $out_dir
    os.mkdir(out_dir)
    sp = SpectrogramParser(audio_config)
    index = 0
    for file in listdir(directory):
        filepath = path.join(directory, file)
        for i in range(5):
            spect,time = sp.parse_audio(filepath)
            name, wav = file.split('.')
            outfile = path.join(out_dir,name+'_{:.2f}'.format(time)+'.'+wav)
            sp.spectrum2wav(spect, sr, outfile)
            specfile = path.join(pos_dir, str(index)+'_'+name+'_{:.2f}'.format(time))
            np.save(specfile,spect)
            index +=1
    # 16000 465984

train_dataset = SpectrogramDataset(audio_conf=audio_config,
                                   pos_dir=pos_dir,
                                   neg_dir=neg_dir,
                                   normalize=False,
                                   speed_volume_perturb=speed_volume_perturb,
                                   spec_augment=spec_augment)
dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=None,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None)


In [6]:
for epoch in range(5):
    for i, (data) in enumerate(dataloader):
        pos_sample, neg_sample = data
        print(pos_sample[0][0][:5],pos_sample.shape)
        break

5
